## 7) 글자 단위 RNN(Char RNN)

지금까지 배운 RNN은 전부 입력과 출력의 단위가 단어 벡터였다. 하지만 입출력의 단위를 단어 레벨(word-level)에서 글자 레벨(character-level)로 변경하여 RNN을 구현할 수 있다. 어려운 내용이 아니라 입, 출력의 단위를 단어에서 글자로 바꿨을 뿐이다.

<img src = 'https://wikidocs.net/images/page/48649/char_rnn1.PNG' width = 60%>

이번 챕터에서는 케라스(Keras)를 사용하여 글자 단위 RNN(Char RNN)을 구현해보겠다.


### 1.글자 단위 RNN 언어 모델(Char RNNLM)

이전 시점의 예측 글자를 다음 시점의 입력으로 사용하는 글자 단위 RNN 언어 모델을 구현해보자. 앞서 배운 단어 단위 RNN 언어 모델과 다른 점은 단어 단위가 아니라 글자 단위를 입, 출력으로 사용하므로 임베딩층(embedding layer)을 여기서는 사용하지 않겠다. 여기서는 언어 모델의 훈련 과정과 테스트 과정의 차이를 이해하는데 초점을 둔다.

다운로드 링크 : http://www.gutenberg.org/files/11/11-0.txt

고전 소설들은 저작권에 보호받지 않으므로, 무료로 쉽게 다운받을 수 있는 좋은 훈련 데이터이다. 위의 링크에서 '이상한 나라의 앨리스(Alice’s Adventures in Wonderland)'라는 소설을 다운로드 한다. 우선, 파일을 불러오고 간단한 전처리를 수행한다.


#### 1) 데이터에 대한 이해와 전처리

In [4]:
import numpy as np
import urllib.request
from tensorflow.keras.utils import to_categorical

In [5]:
urllib.request.urlretrieve('http://www.gutenberg.org/files/11/11-0.txt', 
                           filename = '11-0.txt')
f = open('11-0.txt', 'rb')
lines = []
for line in f: # 데이터를 한 줄씩 읽는다.
    line = line.strip() # strip()을 통해 \r, \n을 제거한다.
    line = line.lower() # 소문자화.
    line = line.decode('ascii', 'ignore') # \xe2\x80\x99 등과 같은 바이트 열 제거

    if len(line) > 0:
        lines.append(line)
f.close()

간단한 전처리가 수행된 결과가 lines란 이름의 리스트에 저장되었다. 리스트에서 5개의 원소만 출력해보겠다.

In [7]:
lines[:5]

['the project gutenberg ebook of alices adventures in wonderland, by lewis carroll',
 'this ebook is for the use of anyone anywhere in the united states and most',
 'other parts of the world at no cost and with almost no restrictions',
 'whatsoever.  you may copy it, give it away or re-use it under the terms of',
 'the project gutenberg license included with this ebook or online at']

각 원소는 문자열로 구성되어져 있는데, 특별히 의미있게 문장 토큰화가 된 상태는 아니다. 이를 하나의 문자열로 통합하겠다.

In [8]:
text = ' '.join(lines)
print('문자열의 길이 또는 총 글자의 개수: %d' % len(text))

문자열의 길이 또는 총 글자의 개수: 159821


하나의 문자열로 통합되었고, 문자열의 길이는 약 15만 9천이다. 일부 출력해보겠다.

In [9]:
print(text[:200])

the project gutenberg ebook of alices adventures in wonderland, by lewis carroll this ebook is for the use of anyone anywhere in the united states and most other parts of the world at no cost and with


이 문자열은 어떤 글자들로 구성되어져 있을까? 이제 이 문자열로부터 글자 집합을 만들겠다. 기존에는 중복을 제거한 단어들의 모음인 단어 집합(vocabulary)을 만들었으나, 이번에 만들 집합은 단어 집합이 아니라 글자 집합이다.

In [10]:
char_vocab = sorted(list(set(text)))
vocab_size = len(char_vocab)
print('글자 집합의 크기 : {}'.format(vocab_size))

글자 집합의 크기 : 57


영어가 훈련 데이터일 때 대부분의 경우에서 글자 집합의 크기가 단어 집합을 사용했을 경우보다 집합의 크기가 현저히 작다는 특징이 있다. 아무리 훈련 코퍼스에 수십만 개 이상의 많은 영어 단어가 존재한다고 하더라도, 영어 단어를 표현하기 위해서 글자 집합에 포함되는 글자는 26개의 알파벳뿐이기 때문이다. 만약 훈련 데이터의 알파벳이 대, 소문자가 구분된 상태라고 하더라도 모든 영어 단어는 총 52개의 알파벳으로 표현 가능하다.

어떤 방대한 양의 텍스트라도 집합의 크기를 적게 가져올 수 있다는 것은 구현과 테스트를 굉장히 쉽게 할 수 있다는 이점을 가지므로, RNN의 동작 메커니즘 이해를 위한 토이 프로젝트로 굉장히 많이 사용된다. 글자 집합에 인덱스를 부여하고 전부 출력해보겠다.

In [11]:
char_to_index = dict((c, i) for i, c in enumerate(char_vocab)) # 글자에 고유한 정수 인덱스 부여
print(char_to_index)

{' ': 0, '!': 1, '"': 2, '#': 3, '$': 4, '%': 5, "'": 6, '(': 7, ')': 8, '*': 9, ',': 10, '-': 11, '.': 12, '/': 13, '0': 14, '1': 15, '2': 16, '3': 17, '4': 18, '5': 19, '6': 20, '7': 21, '8': 22, '9': 23, ':': 24, ';': 25, '?': 26, '@': 27, '[': 28, ']': 29, '_': 30, 'a': 31, 'b': 32, 'c': 33, 'd': 34, 'e': 35, 'f': 36, 'g': 37, 'h': 38, 'i': 39, 'j': 40, 'k': 41, 'l': 42, 'm': 43, 'n': 44, 'o': 45, 'p': 46, 'q': 47, 'r': 48, 's': 49, 't': 50, 'u': 51, 'v': 52, 'w': 53, 'x': 54, 'y': 55, 'z': 56}


인덱스 0부터 28까지는 공백을 포함한 각종 구두점, 특수문자가 존재하고, 인덱스 29부터 54까지는 a부터 z까지 총 26개의 알파벳 소문자가 글자 집합에 포함되어져 있다. 이제 반대로 인덱스로부터 글자를 리턴하는 index_to_char을 만든다.

In [12]:
index_to_char = {}
for key, value in char_to_index.items():
    index_to_char[value] = key

훈련 데이터를 구성해보겠다. 훈련 데이터 구성을 위한 간소화 된 예를 들어보겠다. 훈련 데이터에 apple이라는 시퀀스가 있고, 입력 시퀀스의 길이. 즉, 샘플의 길이를 4라고 한다면 입력 시퀀스와 예측해야 하는 출력 시퀀스는 다음과 같이 구성된다.

``` python
# Example) 샘플의 길이가 4라면 4개의 입력 글자 시퀀스로 부터 4개의 출력 글자 시퀀스를 예측. 즉, RNN의 time step은 4번
appl -> pple
# appl은 train_X(입력 시퀀스), pple는 train_y(예측해야하는 시퀀스)에 저장한다.
```

이제 15만 9천의 길이를 가진 text 문자열로부터 다수의 문장 샘플들로 분리해보겠다. 분리하는 방법은 문장 샘플의 길이를 정하고, 해당 길이만큼 문자열 전체를 전부 등분하는 것이다.



In [13]:
seq_length = 60 # 문장의 길이를 60으로 한다.
n_samples = int(np.floor((len(text) - 1) / seq_length)) # 문자열을 60등분한다. 그러면 즉, 총 샘플의 개수
print('문장 샘플의 수 : {}'.format(n_samples))

문장 샘플의 수 : 2663


만약 문장의 길이를 60으로 한다면 15만 9천을 60으로 나눈 수가 샘플의 수가 된다. 여기서는 총 샘플의 수가 2663개이다.

In [15]:
train_X = []
train_y = []

for i in range(n_samples): # 2663번 수행
    X_sample = text[i * seq_length: (i + 1) * seq_length]
    # 0:60 -> 60:120 -> 120:180로 loop를 돌면서 문장 샘플을 1개씩 가져온다.

    X_encoded = [char_to_index[c] for c in X_sample]
    # 하나의 문장 샘플에 대해서 정수 인코딩

    train_X.append(X_encoded)

    y_sample = text[i * seq_length + 1: (i + 1) * seq_length + 1] # 오른쪽으로 1칸 쉬프트한다.
    
    y_encoded = [char_to_index[c] for c in y_sample]
    train_y.append(y_encoded)

train_X와 train_y의 첫번째 샘플과 두번째 샘플을 출력하여 데이터의 구성을 확인해보자.

In [16]:
print(train_X[0])

[50, 38, 35, 0, 46, 48, 45, 40, 35, 33, 50, 0, 37, 51, 50, 35, 44, 32, 35, 48, 37, 0, 35, 32, 45, 45, 41, 0, 45, 36, 0, 31, 42, 39, 33, 35, 49, 0, 31, 34, 52, 35, 44, 50, 51, 48, 35, 49, 0, 39, 44, 0, 53, 45, 44, 34, 35, 48, 42, 31]


In [17]:
print(train_y[0])

[38, 35, 0, 46, 48, 45, 40, 35, 33, 50, 0, 37, 51, 50, 35, 44, 32, 35, 48, 37, 0, 35, 32, 45, 45, 41, 0, 45, 36, 0, 31, 42, 39, 33, 35, 49, 0, 31, 34, 52, 35, 44, 50, 51, 48, 35, 49, 0, 39, 44, 0, 53, 45, 44, 34, 35, 48, 42, 31, 44]


train_y[0]은 train_X[0]에서 오른쪽으로 한 칸 쉬프트 된 문장임을 알 수 있다.

In [18]:
print(train_X[1])

[44, 34, 10, 0, 32, 55, 0, 42, 35, 53, 39, 49, 0, 33, 31, 48, 48, 45, 42, 42, 0, 50, 38, 39, 49, 0, 35, 32, 45, 45, 41, 0, 39, 49, 0, 36, 45, 48, 0, 50, 38, 35, 0, 51, 49, 35, 0, 45, 36, 0, 31, 44, 55, 45, 44, 35, 0, 31, 44, 55]


In [19]:
print(train_y[1])

[34, 10, 0, 32, 55, 0, 42, 35, 53, 39, 49, 0, 33, 31, 48, 48, 45, 42, 42, 0, 50, 38, 39, 49, 0, 35, 32, 45, 45, 41, 0, 39, 49, 0, 36, 45, 48, 0, 50, 38, 35, 0, 51, 49, 35, 0, 45, 36, 0, 31, 44, 55, 45, 44, 35, 0, 31, 44, 55, 53]


마찬가지로 train_y[1]은 train_X[1]에서 오른쪽으로 한 칸 쉬프트된 문장임을 알 수 있다. 이제 train_X와 train_y에 대해서 원-핫 인코딩을 수행한다. 글자 단위 RNN에서는 입력 시퀀스에 대해서 워드 임베딩을 하지 않는다. 다시 말해 임베딩층(embedding layer)을 사용하지 않을 것이므로, 입력 시퀀스인 train_X에 대해서도 원-핫 인코딩을 한다.

In [20]:
train_X = to_categorical(train_X)
train_y = to_categorical(train_y)

In [21]:
print('train_X의 크기(shape) : {}'.format(train_X.shape)) # 원-핫 인코딩
print('train_y의 크기(shape) : {}'.format(train_y.shape)) # 원-핫 인코딩

train_X의 크기(shape) : (2663, 60, 57)
train_y의 크기(shape) : (2663, 60, 57)


train_X와 train_y의 크기는 2663 x 60 x 57 이다.

<img src = 'https://wikidocs.net/images/page/22886/rnn_image6between7.PNG' width = 60%>

이는 샘플의 수(No. of samples)가 2663개, 입력 시퀀스의 길이(input_length)가 60, 각 벡터의 차원(input_dim)이 57임을 의미한다. 원-핫 벡터의 차원은 글자 집합의 크기인 55이어야 하므로 원-핫 인코딩이 수행되었음을 알 수 있다.

#### 2) 모델 설계하기

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, TimeDistributed

In [24]:
model = Sequential()
model.add(LSTM(256, input_shape = (None, train_X.shape[2]), return_sequences = True))
model.add(LSTM(256, return_sequences = True))
model.add(TimeDistributed(Dense(vocab_size, activation = 'softmax')))

In [25]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])
model.fit(train_X, train_y, epochs = 80, verbose = 2)

Epoch 1/80
84/84 - 42s - loss: 3.0635 - accuracy: 0.1840
Epoch 2/80
84/84 - 40s - loss: 2.7981 - accuracy: 0.2290
Epoch 3/80
84/84 - 39s - loss: 2.4514 - accuracy: 0.3151
Epoch 4/80
84/84 - 39s - loss: 2.3059 - accuracy: 0.3486
Epoch 5/80
84/84 - 39s - loss: 2.2110 - accuracy: 0.3714
Epoch 6/80
84/84 - 39s - loss: 2.1315 - accuracy: 0.3919
Epoch 7/80
84/84 - 39s - loss: 2.0695 - accuracy: 0.4059
Epoch 8/80
84/84 - 38s - loss: 2.0151 - accuracy: 0.4179
Epoch 9/80
84/84 - 39s - loss: 1.9697 - accuracy: 0.4300
Epoch 10/80
84/84 - 39s - loss: 1.9298 - accuracy: 0.4404
Epoch 11/80
84/84 - 40s - loss: 1.8923 - accuracy: 0.4501
Epoch 12/80
84/84 - 39s - loss: 1.8564 - accuracy: 0.4601
Epoch 13/80
84/84 - 39s - loss: 1.8228 - accuracy: 0.4695
Epoch 14/80
84/84 - 39s - loss: 1.7930 - accuracy: 0.4786
Epoch 15/80
84/84 - 39s - loss: 1.7624 - accuracy: 0.4860
Epoch 16/80
84/84 - 39s - loss: 1.7355 - accuracy: 0.4936
Epoch 17/80
84/84 - 39s - loss: 1.7107 - accuracy: 0.4999
Epoch 18/80
84/84 - 39s

In [30]:
def sentence_generation(model, length):

    ix = [np.random.randint(vocab_size)] # 글자에 대한 랜덤 인덱스 생성
    y_char = [index_to_char[ix[-1]]] # 랜덤 인덱스로부터 글자 생성

    print(ix[-1], '번 글자', y_char[-1], '로 예측을 시작!')
    
    X = np.zeros((1, length, vocab_size)) 
    # (1, length, 55) 크기의 X 생성. 즉, LSTM의 입력 시퀀스 생성

    for i in range(length):
        X[0][i][ix[-1]] = 1 # X[0][i][예측한 글자의 인덱스] = 1, 즉, 예측 글자를 다음 입력 시퀀스에 추가

        print(index_to_char[ix[-1]], end = '')
        ix = np.argmax(model.predict(X[:, :i+1, :])[0], 1)
        y_char.append(index_to_char[ix[-1]])
    return ('').join(y_char)

In [31]:
sentence_generation(model, 100)

50 번 글자 t 로 예측을 시작!
the march hare will be much the worldway, said the mock turtle in a large closs, andrally ranating w

'the march hare will be much the worldway, said the mock turtle in a large closs, andrally ranating wh'

### 2.글자 단위 RNN(Char RNN)으로 텍스트 생성하기

이번에는 다 대 일(many-to-one) 구조의 RNN을 글자 단위로 학습시키고, 텍스트 생성을 해보겠다.

#### 1) 데이터에 대한 이해와 전처리

In [33]:
import numpy as np
from tensorflow.keras.utils import to_categorical

다음과 같이 임의로 만든 엉터리 노래 가사가 있다.

In [34]:
text='''
I get on with life as a programmer,
I like to contemplate beer.
But when I start to daydream,
My mind turns straight to wine.

Do I love wine more than beer?

I like to use words about beer.
But when I stop my talking,
My mind turns straight to wine.

I hate bugs and errors.
But I just think back to wine,
And I'm happy once again.

I like to hang out with programming and deep learning.
But when left alone,
My mind turns straight to wine.
'''

우선 위의 텍스트에 존재하는 단락 구분을 없애고 하나의 문자열로 재저장하겠다.

In [35]:
tokens = text.split() # '\n' 제거
text = ' '.join(tokens)
print(text)

I get on with life as a programmer, I like to contemplate beer. But when I start to daydream, My mind turns straight to wine. Do I love wine more than beer? I like to use words about beer. But when I stop my talking, My mind turns straight to wine. I hate bugs and errors. But I just think back to wine, And I'm happy once again. I like to hang out with programming and deep learning. But when left alone, My mind turns straight to wine.


단락 구분이 없어지고 하나의 문자열로 재저장된 것을 확인할 수 있다. 이제 이로부터 글자 집합을 만들어보겠다. 기존에는 중복을 제거한 단어들의 모음인 단어 집합(vocabulary)을 만들었으나, 이번에 만들 집합은 단어 집합이 아니라 글자 집합이다.

In [36]:
char_vocab = sorted(list(set(text))) # 중복을 제거한 글자 집합 생성

print(char_vocab)

[' ', "'", ',', '.', '?', 'A', 'B', 'D', 'I', 'M', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y']


기존의 단어 단위의 집합이 아니라 알파벳 또는 구두점 등의 단위의 집합인 글자 집합이 생성되었다.


In [37]:
vocab_size = len(char_vocab)
print('글자 집합의 크기 : {}'.format(vocab_size))

글자 집합의 크기 : 33


글자 집합의 크기는 33이다.

In [38]:
char_to_index = dict((c, i) for i, c in enumerate(char_vocab)) # 글자에 고유한 정수 인덱스 부여
print(char_to_index)

{' ': 0, "'": 1, ',': 2, '.': 3, '?': 4, 'A': 5, 'B': 6, 'D': 7, 'I': 8, 'M': 9, 'a': 10, 'b': 11, 'c': 12, 'd': 13, 'e': 14, 'f': 15, 'g': 16, 'h': 17, 'i': 18, 'j': 19, 'k': 20, 'l': 21, 'm': 22, 'n': 23, 'o': 24, 'p': 25, 'r': 26, 's': 27, 't': 28, 'u': 29, 'v': 30, 'w': 31, 'y': 32}


이번 실습의 글자 집합의 경우 훈련 데이터에 등장한 알파벳의 대, 소문자를 구분하고 구두점과 공백을 포함하였다. 이제 훈련에 사용할 문장 샘플들을 만들어보겠다. 여기서는 RNN을 이용하여 생성한 텍스트 챕터와 유사하게 데이터를 구성한다. 다만, 단위가 글자 단위라는 점이 다르다. 예를 들어 훈련 데이터에 student라는 단어가 있고, 입력 시퀀스의 길이를 5라고 한다면 입력 시퀀스와 예측해야하는 글자는 다음과 같이 구성된다.

``` python
# Example) 5개의 입력 글자 시퀀스로부터 다음 글자 시퀀스를 예측. 즉, RNN의 time step은 5번
stude -> n 
tuden -> t
```

여기서는 입력 시퀀스의 길이. 즉, 모든 샘플들의 길이가 10이 되도록 데이터를 구성해보겠다. 예측 대상이 되는 글자도 필요하므로 우선 길이가 11이 되도록 데이터를 구성한다.

In [39]:
length = 11
sequences = []
for i in range(length, len(text)):
    seq = text[i-length:i] # 길이 11의 문자열을 지속적으로 만든다.

    sequences.append(seq)
print('총 훈련 샘플의 수: %d' % len(sequences))

총 훈련 샘플의 수: 426


총 샘플의 수는 426개가 완성되었다. 이 중 10개만 출력해보겠다.

In [40]:
sequences[:10]

['I get on wi',
 ' get on wit',
 'get on with',
 'et on with ',
 't on with l',
 ' on with li',
 'on with lif',
 'n with life',
 ' with life ',
 'with life a']

첫번째 문장이었던 'I get on with life as a programmer,'가 10개의 샘플로 분리된 것을 확인할 수 있다. 다른 문장들에 대해서도 sequences에 모두 저장되어져 있다. 원한다면, sequences[30:45] 등과 같이 인덱스 범위를 변경하여 출력해보기 바란다. 이제 앞서 만든 char_to_index를 사용하여 전체 데이터에 대해서 정수 인코딩을 수행한다.

In [41]:
X = []
for line in sequences:
    temp_X = [char_to_index[char] for char in line]
    X.append(temp_X)

정수 인코딩 된 결과가 X에 저장되었다. 5개만 출력해보겠다.

In [42]:
for line in X[:5]:
    print(line)

[8, 0, 16, 14, 28, 0, 24, 23, 0, 31, 18]
[0, 16, 14, 28, 0, 24, 23, 0, 31, 18, 28]
[16, 14, 28, 0, 24, 23, 0, 31, 18, 28, 17]
[14, 28, 0, 24, 23, 0, 31, 18, 28, 17, 0]
[28, 0, 24, 23, 0, 31, 18, 28, 17, 0, 21]


정상적으로 정수 인코딩이 수행되었다. 이제 예측 대상인 글자를 분리시켜주는 작업을 한다. 모든 샘플 문장에 대해서 맨 마지막 글자를 분리시켜준다.

In [43]:
sequences = np.array(X)
X = sequences[:, :-1]
y = sequences[:, -1] # 맨 마지막 위치의 글자를 분리

정상적으로 분리가 되었는지 X와 y 둘 다 5개씩 출력해보겠다.

In [44]:
for line in X[:5]:
    print(line)

[ 8  0 16 14 28  0 24 23  0 31]
[ 0 16 14 28  0 24 23  0 31 18]
[16 14 28  0 24 23  0 31 18 28]
[14 28  0 24 23  0 31 18 28 17]
[28  0 24 23  0 31 18 28 17  0]


In [45]:
print(y[:5])

[18 28 17  0 21]


앞서 출력한 5개의 샘플에서 각각 맨 뒤의 글자였던 18, 28, 17, 0, 21이 별도로 분리되어 y에 저장되었다. 이제 X와 y에 대해서 원-핫 인코딩을 수행해보겠다.

In [46]:
sequences = [to_categorical(x, num_classes = vocab_size) for x in X] # X에 대한 원-핫 인코딩
X = np.array(sequences)
y = to_categorical(y, num_classes = vocab_size) # y에 대한 원-핫 인코딩

원-핫 인코딩이 수행되었는지 확인하기 위해 수행한 후의 X의 크기(shape)를 보겠다.

In [47]:
print(X.shape)

(426, 10, 33)


현재 X의 크기는 426 x 10 x 33 이다.

<img src = 'https://wikidocs.net/images/page/22886/rnn_image6between7.PNG' width = 60%>

이는 샘플의 수(No. of samples)가 426개, 입력 시퀀스의 길이(input_length)가 10, 각 벡터의 차원(input_dim)이 33임을 의미한다. 원-핫 벡터의 차원은 글자 집합의 크기인 33이어야 하므로 X에 대해서 원-핫 인코딩이 수행되었음을 알 수 있다.

#### 2) 모델 설계하기



In [48]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [49]:
model = Sequential()
model.add(LSTM(80, input_shape = (X.shape[1], X.shape[2]))) # X.shape[1]은 25, X.shape[2]는 33
model.add(Dense(vocab_size, activation = 'softmax'))

LSTM을 사용하고, 은닉 상태의 크기는 80, 그리고 출력층에 단어 집합의 크기만큼의 뉴런을 배치하여 모델을 설계한다.

In [50]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])
model.fit(X, y, epochs = 100, verbose = 2)

Epoch 1/100
14/14 - 2s - loss: 3.4751 - accuracy: 0.0939
Epoch 2/100
14/14 - 0s - loss: 3.3638 - accuracy: 0.1972
Epoch 3/100
14/14 - 0s - loss: 3.0867 - accuracy: 0.1972
Epoch 4/100
14/14 - 0s - loss: 2.9728 - accuracy: 0.1972
Epoch 5/100
14/14 - 0s - loss: 2.9375 - accuracy: 0.1972
Epoch 6/100
14/14 - 0s - loss: 2.9155 - accuracy: 0.1972
Epoch 7/100
14/14 - 0s - loss: 2.8898 - accuracy: 0.1972
Epoch 8/100
14/14 - 0s - loss: 2.8683 - accuracy: 0.1972
Epoch 9/100
14/14 - 0s - loss: 2.8460 - accuracy: 0.1972
Epoch 10/100
14/14 - 0s - loss: 2.8097 - accuracy: 0.2042
Epoch 11/100
14/14 - 0s - loss: 2.7689 - accuracy: 0.2230
Epoch 12/100
14/14 - 0s - loss: 2.7141 - accuracy: 0.2300
Epoch 13/100
14/14 - 0s - loss: 2.6670 - accuracy: 0.2700
Epoch 14/100
14/14 - 0s - loss: 2.6237 - accuracy: 0.2347
Epoch 15/100
14/14 - 0s - loss: 2.5779 - accuracy: 0.2582
Epoch 16/100
14/14 - 0s - loss: 2.5362 - accuracy: 0.2746
Epoch 17/100
14/14 - 0s - loss: 2.5074 - accuracy: 0.3005
Epoch 18/100
14/14 - 0s

출력층의 활성화 함수로는 소프트맥스 함수, 손실 함수로는 크로스 엔트로피 함수를 사용하여 총 100번의 에포크를 수행한다.

문장을 생성하는 함수 sentence_generation을 만들어서 문장을 생성해보자.

In [51]:
def sentence_generation(model, char_to_index, seq_length, seed_text, n):
    # 모델, 인덱스 정보, 문장 길이, 초기 시퀀스, 반복 횟수

    init_text = seed_text # 문장 생성에 사용할 초기 시퀀스
    sentence = ''

    for _ in range(n): # n번 반복
        encoded = [char_to_index[char] for char in seed_text] # 현재 시퀀스에 대한 정수 인코딩

        encoded = pad_sequences([encoded], maxlen = seq_length,
                                padding = 'pre') # 데이터에 대한 패딩
        encoded = to_categorical(encoded, num_classes = len(char_to_index))

        result = model.predict_classes(encoded, verbose = 0)
        # 입력한 X(현재 시퀀스)에 대해서 y를 예측하고 y(예측한 글자)를 result에 저장.

        for char, index in char_to_index.items(): # 만약 예측한 글자와 인덱스와 동일한 글자가 있다면
            if index == result: # 해당 글자가 예측 글자이므로 break
                break
        
        seed_text = seed_text + char # 현재 시퀀스 + 예측 글자를 현재 시퀀스로 변경

        sentence = sentence + char # 예측 글자를 문장에 저장
        # for문이므로 이 작업을 다시 반복
    
    sentence = init_text + sentence
    return sentence

In [52]:
print(sentence_generation(model, char_to_index, 10, 'I get on w', 80))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


I get on with life as a programmer, I like to hang out with programming and deep learning.


두 개의 문장이 출력되었는데 훈련 데이터에서는 연속적으로 나온 적이 없는 두 문장임에도 모델이 임의로 잘 생성해낸 것 같다.